In [ ]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.insert(0, '../')
sys.path.insert(0, '../python_src/')

import numpy as np
import numpy.random as rand
import scipy as sp
import gzip
import pickle
import itertools as it
import shelve
import glob

import network_generation as ngen

import matplotlib.pyplot as plt; plt.rcdefaults()
import seaborn as sns
%matplotlib inline
import matplotlib as mpl
import matplotlib.colors as mcolors
from matplotlib import collections as mc
import network_plot as nplot

In [ ]:
# DIM = 2
# NN = 2**6
# Lp = -1.0
# label = "2d_harmonic/poly_uniform/state_N{:08d}_Lp{:.4f}".format(NN, Lp)

DIM = 3
NN = 2**9
Lp = -1.0
label = "3d_harmonic/bi/state_N{:08d}_Lp{:.4f}".format(NN, Lp)



src_fn = "/home/rocks/data/network_periodic_jammed/network_{}D_N{:05d}_Lp{:.2f}".format(DIM, NN, Lp)

db_fn = "/home/rocks/data/network_finite_ball/network_{}D_N{:05d}_Lp{:.2f}".format(DIM, NN, Lp)


with shelve.open(db_fn) as db:
    
    for index in range(1024):

        print(index)
        net = ngen.load_network(src_fn, index)
        net = ngen.make_finite(net)
        net = ngen.make_ball(net, 1.0) 
        net = ngen.prune_underconstrained(net)
        
        db["{}".format(index)] = net

In [ ]:
sns.set(context='talk', font_scale=1.0, color_codes=True, palette='deep', style='ticks', 
        rc={'mathtext.fontset': 'cm', 'xtick.direction': 'in','ytick.direction': 'in',
            'axes.linewidth': 1.5, 'figure.dpi':100})

fig = plt.figure(figsize=(4, 4))
ax1 = fig.add_subplot(1, 1, 1, aspect=1.0)
gray = "#bdbdbd"

########################################################3

DIM = 2
NN = 64
Lp = -1.0

db_fn = "/home/rocks/data/network_finite_ball/network_{}D_N{:05d}_Lp{:.2f}".format(DIM, NN, Lp)
index = 0

net = ngen.load_network(db_fn, index)
boundary = net['boundary']

(inodei, inodej) = ngen.choose_bondary_edge(net, 90)

inodesi = [inodei]
inodesj = [inodej]

(onodei, onodej) = ngen.choose_bondary_edge(net, 90+180)

onodesi = [onodei]
onodesj = [onodej]


net = ngen.convert_to_network_object(net, periodic=False)

NV = net.NN
NE = net.NE
edgei = net.edgei
edgej = net.edgej
K = net.K
scale = 1.0

vert_pos = np.array(net.node_pos, float)
L = np.array(net.L, float)

styles = {edge: {} for edge in range(net.NE)}

knorm = mcolors.Normalize(vmin = 0.0, vmax = np.max(K))
for b in range(len(K)):
    styles[b]['lw'] = 2.0*scale*knorm(K[b])
    styles[b]['color'] = gray


nplot.show_network(ax1, net, styles=styles, alpha=1.0)



styles = {}
for node in range(net.NN):    
    styles[node] = {}
    styles[node]['size'] = 10
    
for node in boundary:
    styles[node]['color'] = 'b'
    styles[node]['size'] = 50
    
    
for i in inodesi:
    styles[i] = {}
    styles[i]['color'] = 'r'
for i in inodesj:
    styles[i] = {}
    styles[i]['color'] = 'r'
    
for i in onodesi:
    styles[i] = {}
    styles[i]['color'] = 'g'
for i in onodesj:
    styles[i] = {}
    styles[i]['color'] = 'g'
        
nplot.show_nodes(ax1, net, list(styles.keys()), styles=styles, marker='o')



ax1.get_xaxis().set_ticks([])
ax1.get_yaxis().set_ticks([])

plt.show()